In [50]:
import numpy as np
import os
import pathlib
import random
import torch
import torchgeometry
import cv2
from dlclive import DLCLive, Processor



from torch.utils.data import DataLoader

# potential mass parametrizations
from differentiable_robot_model.rigid_body_params import (
    UnconstrainedScalar,
    PositiveScalar,
    UnconstrainedTensor,
)

# potential inertia matrix parametrizations
from differentiable_robot_model.rigid_body_params import (
    CovParameterized3DInertiaMatrixNet,
    Symm3DInertiaMatrixNet,
    SymmPosDef3DInertiaMatrixNet,
    TriangParam3DInertiaMatrixNet,
)

from differentiable_robot_model.robot_model import (
    DifferentiableRobotModel,
    DifferentiableKUKAiiwa,
)
from differentiable_robot_model.data_utils import (
    generate_sine_motion_forward_dynamics_data,
)
import diff_robot_data

torch.set_printoptions(precision=3, sci_mode=False)

random.seed(0)
np.random.seed(1)
torch.manual_seed(0)


In [51]:
class NMSELoss(torch.nn.Module):
    def __init__(self, var):
        super(NMSELoss, self).__init__()
        self.var = var

    def forward(self, yp, yt):
        err = (yp - yt) ** 2
        werr = err / self.var
        return werr.mean()


urdf_path = os.path.join("a1.urdf")
device = "cpu"
learnable_robot_model = DifferentiableRobotModel(
    urdf_path, "A1", device=device
)

learnable_robot_model.make_link_param_learnable(
    "FR_hip", "trans", UnconstrainedTensor(dim1=1, dim2=3)
)
learnable_robot_model.make_link_param_learnable(
        "FR_thigh_shoulder", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "FR_thigh", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "FR_calf", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "FR_foot", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "FL_hip", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "FL_thigh_shoulder", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "FL_thigh", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "FL_calf", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "FL_foot", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "RR_hip", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "RR_thigh_shoulder", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "RR_thigh", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "RR_calf", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "RR_foot", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "RL_hip", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "RL_thigh_shoulder", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "RL_thigh", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "RL_calf", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )
learnable_robot_model.make_link_param_learnable(
        "RL_foot", "trans", UnconstrainedTensor(dim1=1, dim2=3)
    )

# learnable_robot_model.print_learnable_params()
joint_angles = torch.rand((1, 12), requires_grad=True)
learnable_robot_model.compute_forward_kinematics(joint_angles,"FR_foot")

Unknown tag "hardwareInterface" in /robot[@name='a1']/transmission[@name='FR_hip_tran']/actuator[@name='FR_hip_motor']
Unknown tag "hardwareInterface" in /robot[@name='a1']/transmission[@name='FR_thigh_tran']/actuator[@name='FR_thigh_motor']
Unknown tag "hardwareInterface" in /robot[@name='a1']/transmission[@name='FR_calf_tran']/actuator[@name='FR_calf_motor']
Unknown tag "hardwareInterface" in /robot[@name='a1']/transmission[@name='FL_hip_tran']/actuator[@name='FL_hip_motor']
Unknown tag "hardwareInterface" in /robot[@name='a1']/transmission[@name='FL_thigh_tran']/actuator[@name='FL_thigh_motor']
Unknown tag "hardwareInterface" in /robot[@name='a1']/transmission[@name='FL_calf_tran']/actuator[@name='FL_calf_motor']
Unknown tag "hardwareInterface" in /robot[@name='a1']/transmission[@name='RR_hip_tran']/actuator[@name='RR_hip_motor']
Unknown tag "hardwareInterface" in /robot[@name='a1']/transmission[@name='RR_thigh_tran']/actuator[@name='RR_thigh_motor']
Unknown tag "hardwareInterface" 

(tensor([[ 0.027,  0.103, -0.338]], grad_fn=<AddBackward0>),
 tensor([[0.315, 0.360, 0.131, 0.868]], grad_fn=<CopySlices>))

In [74]:
keypoints = """End of Neck
Shoulder
FL_Knee
FL_Ankle
FL_White_TapeTop
FL_White_TapeBot
FR_Knee
FR_Ankle
FL_Red_TapeTop
FL_Red_TapeBot
End of Tail
Hip
BL_Knee
BL_Ankle
BL_Red_TapeTop
BL_Red_TapeBot
BR_Knee
BR_Ankle
BR_Red_TapeTop
BR_Red_TapeBot""".split("\n")
training_keypoints = ['FL_Ankle','FL_Knee','BL_Ankle','BL_Knee']
indices_keypoints_training = [keypoints.index(val) for val in training_keypoints]

base_dir = pathlib.Path.cwd()
img_path = base_dir / "HorseInferenceFiles/img0088.png"
dlc_model_path = base_dir/"HorseInferenceFiles/DLC_HorseProject1_efficientnet-b0_iteration-0_shuffle-1"
image = cv2.imread(str(img_path))
dlc_proc = Processor()
dlc_live = DLCLive(model_path=str(dlc_model_path), processor=dlc_proc)

out = list()
for ind, image in enumerate([cv2.imread(str(base_dir / ("HorseInferenceFiles/"+filePath))) for filePath in ["img0088.png","img0228.png","img0288.png","img0484.png"]]):
    pose = None
    if ind == 0:
        pose = (dlc_live.init_inference(image)[indices_keypoints_training][:,0:2])
    else:
        pose = (dlc_live.get_pose(image)[indices_keypoints_training][:,0:2])
    pose = torch.from_numpy(pose)
    out.append(pose)

# dlc_live.init_inference(image)
# img_keypoints = dlc_live.get_pose(image)

# training_data = img_keypoints[indices_keypoints_training]
# training_data = training_data[:,0:2]
# training_data

training_data = out = torch.stack(out)
training_data

tensor([[[275.799, 244.884],
         [249.626, 216.098],
         [340.429, 246.946],
         [349.002, 210.681]],

        [[206.467, 256.802],
         [215.914, 218.328],
         [366.530, 252.797],
         [373.324, 215.444]],

        [[249.048, 258.187],
         [243.911, 220.433],
         [410.155, 250.348],
         [389.190, 213.207]],

        [[252.591, 246.068],
         [225.266, 209.619],
         [339.606, 259.536],
         [349.615, 220.834]]])

In [54]:
cameraLoss = torch.sum(torch.sqrt(torch.sum(torch.pow(training_data-cameraEstimate, 2), dim = 1)))
cameraLoss

tensor(4349.010, grad_fn=<SumBackward0>)

In [77]:
joint_angles_array = torch.rand((out.size(0), 1, 12), requires_grad=True)

intrinsic = (100*torch.eye(4).unsqueeze(0)).requires_grad_(True)
extrinsic = (torch.ones((1,4,4))*100).requires_grad_(True)
depth = torch.ones((4,1,1,4), requires_grad=False) #Without depth, loss doesnt descrease. with depth loss rapidly decreases.
intrinsic.retain_grad()
optimizer = torch.optim.Adam([joint_angles_array, intrinsic, extrinsic] + list(learnable_robot_model.parameters()), lr=1e-2)
for epoch in range(2000):
    optimizer.zero_grad()

    projections = []
    #Where the Robot Thinks It Is
    for joint_angles in joint_angles_array:
        projection = torch.cat((learnable_robot_model.compute_forward_kinematics(joint_angles, "FL_foot")[0],\
        learnable_robot_model.compute_forward_kinematics(joint_angles, "FL_calf")[0],\
        learnable_robot_model.compute_forward_kinematics(joint_angles, "RR_foot")[0],\
        learnable_robot_model.compute_forward_kinematics(joint_angles, "RR_calf")[0]))
        projection = projection.unsqueeze(0).unsqueeze(0)
        projections.append(projection)
    projection = torch.cat(projections)

    #Where Robot Is In Pixel Space
    cameraModel = torchgeometry.PinholeCamera(intrinsic, extrinsic, imageHeight, torch.tensor([1,]))
    cameraEstimate = torchgeometry.pixel2cam(depth, torch.cat([cameraModel.intrinsics_inverse() for _ in range(4)]), projection)
    #Remove the 3rd Column of Results Since Unneeded
    cameraEstimate = cameraEstimate[:,:,:,0:2]
    #Removes the heigh dimension from the results since pixels not organied into grid
    cameraEstimate = cameraEstimate.squeeze(1)

    #Loss Between Where Robot/Horse Should Be and Where It Is
    loss = torch.sum(torch.sqrt(torch.sum(torch.pow((training_data)-cameraEstimate, 2))))
    if epoch % 100 == 0:
        print(f"Epoch {epoch}: Loss ({loss})")
    loss.backward()
    optimizer.step()


Epoch 0: Loss (1523.9345703125)
Epoch 100: Loss (1466.9132080078125)
